`xt = tx1 + (1 − t)x0 `              

 This equation represents a linear interpolation between two points x0 and x1 using a parameter t.

The variable xt is the interpolated point between x0 and x1. t is the interpolation parameter that ranges from 0 to 1.

When t=0, xt = x0, meaning xt is equal to the first point x0. 

When t=1, xt = x1, meaning xt is equal to the second point x1.

For values of t between 0 and 1, xt is a linear combination of x0 and x1. Specifically, xt is equal to x0 multiplied by (1 - t) plus x1 multiplied by t. 

As t increases from 0 to 1, the interpolated point xt moves linearly from x0 to x1 along the line segment between those two points. This allows you to find intermediate points along a straight line path between x0 and x1.

In summary, this equation performs linear interpolation between two endpoints based on a parameter t. By varying t from 0 to 1, xt moves uniformly along the straight line path between x0 and x1.


`xt ∼ N (tx1, (1 − t)2I),`

 This notation represents a normal (Gaussian) distribution with the following properties:

- xt is a random vector of variables that is normally distributed.
- The mean of xt is tx1, where t is a scalar and x1 is a fixed vector. So the mean scales linearly with t.
- The covariance matrix is (1 - t)2I, where I is the identity matrix. So the covariance scales quadratically with (1 - t). 
- As t increases, the variance decreases quadratically towards 0. When t=1, the variance is 0 and xt collapses to the fixed vector x1.
- As t decreases towards 0, the variance increases quadratically, spreading out the distribution.

In summary, xt interpolates between a fixed vector x1 and a normal distribution with increasing variance as t goes from 1 to 0. The parameter t controls how close xt sticks to the fixed mean x1 versus being spread out.


`xt = tx1 ⊕ (1 − t) Down(x0, 2K ),`

 This equation is calculating a new image xt by combining two existing images x0 and x1 using a weighted average.

Specifically:

- tx1 represents image x1 weighted by a factor t. So if t=0.5, this would be 0.5 * x1.

- (1 - t) Down(x0, 2K) represents image x0 downscaled by a factor of 2K (reducing resolution), weighted by (1 - t). So if t=0.5, this would be 0.5 * Down(x0, 2K). 

- The ⊕ symbol represents element-wise addition of the two weighted images. 

So in summary, xt is computed by taking a weighted average of x1 and a downscaled version of x0, controlled by the weighting factor t. This has the effect of transitioning smoothly from x0 to x1 as t goes from 0 to 1, while matching resolutions by downscaling x0. The downscaling of x0 avoids artifacts when combining images of different sizes.


`xt = t′ Down(xek , 2k) + (1 − t′) Up(Down(xsk , 2k+1)),`


 This equation defines xt as a weighted combination of two downsampled images xek and xsk.

Specifically:
- xek is downsampled by a factor of 2k 
- xsk is downsampled by a factor of 2k+1 and then upsampled back to the original size
- t' is a weighting factor between 0 and 1
- xt is computed as:
    - t' * Downsampled xek 
    + (1 - t') * Upsampled downsampled xsk

So xt is a mixture of a mildly and heavily downsampled version of the input images, with relative weighting determined by t'. This has the effect of producing a smoothed/blurred combination of the inputs.

In [2]:
import torch
import torch.nn as nn

# Define a 2D convolutional layer with dilation
conv_layer = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, dilation=2)
print(conv_layer)

# Input tensor of shape (batch_size, channels, height, width)
input_tensor = torch.randn(1, 1, 10, 10)
print(input_tensor)

# Apply the convolutional layer
output_tensor = conv_layer(input_tensor)

print(output_tensor.shape)


Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), dilation=(2, 2))
tensor([[[[-2.8445e+00,  5.1145e-01, -1.2006e+00, -1.6100e+00,  5.1081e-01,
           -1.4714e+00, -8.5396e-01,  4.5243e-01,  1.2125e+00,  1.1608e+00],
          [-5.6500e-01,  7.6027e-01, -8.7526e-01, -1.3108e-02,  2.3209e-01,
            8.2394e-01,  3.0478e-01,  1.1288e+00,  7.6737e-01,  1.7017e+00],
          [-2.1264e-01, -1.8750e+00, -2.1112e+00,  2.4271e-01, -5.3580e-01,
            5.8242e-01, -9.6039e-01,  2.9491e-01,  4.2169e-01, -2.5280e-01],
          [-3.3099e-01, -7.0068e-01, -2.2981e-01,  1.5504e+00, -4.4323e-01,
            1.3953e+00, -2.0092e+00,  1.4159e-01,  1.7603e+00,  1.3945e+00],
          [ 6.9160e-01,  3.9457e-01, -6.2283e-01, -9.3275e-01,  4.1003e-01,
           -2.0210e+00,  9.8246e-01,  2.0212e-01,  6.6417e-01,  1.0909e+00],
          [ 2.9392e-01, -5.8461e-01,  5.7005e-01,  1.6243e-02, -9.4424e-01,
           -6.2767e-03,  1.1252e+00, -1.2797e+00, -4.2013e-04,  9.2083e-01],
          [ 8.463

In [37]:
import torch 
from torch import nn
from typing import Tuple, Union


class CausalConv3d(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size: int = 3,
        stride: Union[int, Tuple[int]] = 1,
        dilation: int = 1,
        groups: int = 1,
        **kwargs,
    ):
        super().__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        kernel_size = (kernel_size, kernel_size, kernel_size)
        self.time_kernel_size = kernel_size[0]

        dilation = (dilation, 1, 1)

        height_pad = kernel_size[1] // 2
        width_pad = kernel_size[2] // 2
        padding = (0, height_pad, width_pad)

        self.conv = nn.Conv3d(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            dilation=dilation,
            padding=padding,
            padding_mode="zeros",
            groups=groups,
        )

    def forward(self, x, causal: bool = True):
        print("x", x.shape)  # -> ([2, 3, 10, 20, 20]) -> (Batch_size, channels, depth, height, width)
        print("x details: ", x[:, :, :1, :, :].shape)  # ([2, 3, 1, 20, 20])
        
        if causal:
            first_frame_pad = x[:, :, :1, :, :].repeat(
                (1, 1, self.time_kernel_size - 1, 1, 1)   
            )
            print("first frame padding", first_frame_pad.shape) # ([2, 3, 2, 20, 20])
            
            detail_first_frame_pad = x[:, :, :1, :, :]  # ([2, 3, 1, 20, 20])
            detail_first_frame_pad = detail_first_frame_pad.repeat(1, 1, self.time_kernel_size - 1, 1, 1)  #  kernel_size -> 3 - 1 = 2 
            print("detail first frame padding: ", detail_first_frame_pad.shape) # ([2, 3, 2, 20, 20])
            
            x = torch.cat((first_frame_pad, x), dim=2)
            print("x", x.shape)  # ([2, 3, 12, 20, 20]), ([2, 3, 10, 20, 20])
            
            print("first frame padding in x: ", x[0].shape) # ([3, 12, 20, 20])
            print("first frame padding in x1: ", x[1].shape)
           
            
        else:
            first_frame_pad = x[:, :, :1, :, :].repeat(
                (1, 1, (self.time_kernel_size - 1) // 2, 1, 1)
            )
            print(f"first frame padding: {first_frame_pad.shape}")   # ([2, 3, 1, 20, 20])
            
            last_frame_pad = x[:, :, -1:, :, :].repeat(
                (1, 1, (self.time_kernel_size - 1) // 2, 1, 1)
            )
            
            print("last frame padding", last_frame_pad.shape)
            
            
            x = torch.cat((first_frame_pad, x, last_frame_pad), dim=2)
            print("x", x.shape)
        x = self.conv(x)
        return x

    @property
    def weight(self):
        return self.conv.weight


In [38]:
# Sample Input Tensor
batch_size = 2
channels = 3
depth = 10  # Time dimension
height = 20
width = 20

# Random tensor with the shape (batch_size, channels, depth, height, width)
input_tensor = torch.randn(batch_size, channels, depth, height, width)
print("input Tensor: ", input_tensor.shape)

# Instantiate the CausalConv3d layer
causal_conv3d_layer = CausalConv3d(in_channels=channels, out_channels=6, kernel_size=3)
print("causal conv3d layer: ", causal_conv3d_layer)

# Apply the causal convolution with causal=True
output_tensor_causal = causal_conv3d_layer(input_tensor, causal=True)

# Apply the causal convolution with causal=False
output_tensor_non_causal = causal_conv3d_layer(input_tensor, causal=False)

# Print the shapes of the output tensors
print("Output Shape with Causal Padding:", output_tensor_causal.shape)
print("Output Shape with Non-Causal Padding:", output_tensor_non_causal.shape)


input Tensor:  torch.Size([2, 3, 10, 20, 20])
causal conv3d layer:  CausalConv3d(
  (conv): Conv3d(3, 6, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
)
x torch.Size([2, 3, 10, 20, 20])
x details:  torch.Size([2, 3, 1, 20, 20])
first frame padding torch.Size([2, 3, 2, 20, 20])
detail first frame padding:  torch.Size([2, 3, 2, 20, 20])
x torch.Size([2, 3, 12, 20, 20])
first frame padding in x:  torch.Size([3, 12, 20, 20])
first frame padding in x1:  torch.Size([3, 12, 20, 20])
x torch.Size([2, 3, 10, 20, 20])
x details:  torch.Size([2, 3, 1, 20, 20])
first frame padding: torch.Size([2, 3, 1, 20, 20])
last frame padding torch.Size([2, 3, 1, 20, 20])
x torch.Size([2, 3, 12, 20, 20])
Output Shape with Causal Padding: torch.Size([2, 6, 10, 20, 20])
Output Shape with Non-Causal Padding: torch.Size([2, 6, 10, 20, 20])


In [24]:
# formula of dialation: R + (R - 1) x (d - 1)
# R - currnel size 
# d - dilation 

3 + ( 3 - 1)* (2 - 1)


5

## torch.nn.init.kaiming_uniform_

In [2]:
import torch
import torch.nn as nn
import math

weight = torch.empty(3, 5)
print(f"Weight: {weight}")
nn.init.kaiming_uniform_(weight, a=math.sqrt(5))
print(weight)


Weight: tensor([[7.9278e+28, 9.6269e-43, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]])
tensor([[ 0.3333,  0.1480, -0.4159, -0.4272,  0.2004],
        [-0.2514,  0.2156,  0.3105, -0.3438,  0.2856],
        [-0.3329, -0.2017, -0.0799, -0.3891, -0.3182]])


## nn.init._calculate_fan_in_and_fan_out(self.weight1)

In [3]:
import torch
from torch import nn 

# (out_channels, in_channels, depth, height, width)
weight = torch.empty(16, 3, 3, 3, 3)
print("weight", weight)

fan_in, fan_out = nn.init._calculate_fan_in_and_fan_out(weight)
print("Fan-In:", fan_in)
print("Fan-Out:", fan_out)


weight tensor([[[[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]]],


         [[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]]],


         [[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]]]],



        [[[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]]],


         [[[0., 0., 0.],
           [0., 0., 0.],
         

In [5]:
import torch
import torch.nn as nn

# Define a 3D convolutional layer
conv3d_layer = nn.Conv3d(in_channels=3, out_channels=16, kernel_size=(3, 3, 3))

# Print the shape of the weight tensor
print(conv3d_layer.weight.shape)  # Should output torch.Size([16, 3, 3, 3, 3])

# Example input tensor (e.g., a batch of 3D volumes)
input_tensor = torch.randn(1, 3, 32, 32, 32)  # shape (batch_size, in_channels, depth, height, width)

# Apply the convolutional layer to the input tensor
output_tensor = conv3d_layer(input_tensor)

# Print the shape of the output tensor
print(output_tensor.shape)  # Output shape depends on stride, padding, etc.


torch.Size([16, 3, 3, 3, 3])
torch.Size([1, 16, 30, 30, 30])


## how to work inspect.signature()

In [2]:
import inspect
import torch.nn as nn

class ExampleDecoder(nn.Module):
    def forward(self, x, y, z=0):
        pass

# Create an instance of the ExampleDecoder
decoder = ExampleDecoder()

# Get the signature of the forward method
signature = inspect.signature(decoder.forward)

# Get the parameters of the forward method
parameters = signature.parameters

# Print the parameters
for name, param in parameters.items():
    print(f"Parameter name: {name}")
    print(f"  Default value: {param.default}")
    print(f"  Annotation: {param.annotation}")
    print(f"  Kind: {param.kind}")

# This will output:
# Parameter name: x
#   Default value: <class 'inspect._empty'>
#   Annotation: <class 'inspect._empty'>
#   Kind: POSITIONAL_OR_KEYWORD
# Parameter name: y
#   Default value: <class 'inspect._empty'>
#   Annotation: <class 'inspect._empty'>
#   Kind: POSITIONAL_OR_KEYWORD
# Parameter name: z
#   Default value: 0
#   Annotation: <class 'inspect._empty'>
#   Kind: POSITIONAL_OR_KEYWORD


Parameter name: x
  Default value: <class 'inspect._empty'>
  Annotation: <class 'inspect._empty'>
  Kind: POSITIONAL_OR_KEYWORD
Parameter name: y
  Default value: <class 'inspect._empty'>
  Annotation: <class 'inspect._empty'>
  Kind: POSITIONAL_OR_KEYWORD
Parameter name: z
  Default value: 0
  Annotation: <class 'inspect._empty'>
  Kind: POSITIONAL_OR_KEYWORD


## ConfigMixin

In [10]:
from diffusers import ModelMixin, ConfigMixin
import torch.nn as nn
import torch

class ExampleModel(ConfigMixin, nn.Module):
    config_name = "example_model"

    def __init__(self, param1: int = 1, param2: str = "default"):
        super().__init__()
        self.param1 = param1
        self.param2 = param2
        self.layer = nn.Linear(10, 10)

    def forward(self, x):
        return self.layer(x)

    def save_pretrained(self, save_directory):
        # Save the model's state_dict (weights)
        torch.save(self.state_dict(), f"{save_directory}/pytorch_model.bin")
        # Optionally save the configuration
        self.save_config(save_directory)

    @classmethod
    def from_pretrained(cls, load_directory):
        # Load the model's state_dict (weights)
        state_dict = torch.load(f"{load_directory}/pytorch_model.bin")
        # Instantiate the model
        model = cls()
        # Load the state_dict into the model
        model.load_state_dict(state_dict)
        return model

# Example usage to save and load the model
model = ExampleModel(param1=42, param2="example")
model.save_pretrained("D:\\AI\\test\\config_mixin")

# Loading the model
loaded_model = ExampleModel.from_pretrained("D:\\AI\\test\\config_mixin")


C:\Users\Mandhata Kumar\AppData\Local\Temp\ipykernel_9580\3783680460.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f"{load_directory}/pytorch_

In [ ]:
from typing import Optional, Union

import torch
import inspect
import math
import torch.nn as nn
from diffusers import ConfigMixin, ModelMixin
from diffusers.models.autoencoders.vae import (
    DecoderOutput,
    DiagonalGaussianDistribution,
)
from diffusers.models.modeling_outputs import AutoencoderKLOutput
from ltx_video.models.autoencoders.conv_nd_factory import make_conv_nd


class AutoencoderKLWrapper(ModelMixin, ConfigMixin):
    """Variational Autoencoder (VAE) model with KL loss.

    VAE from the paper Auto-Encoding Variational Bayes by Diederik P. Kingma and Max Welling.
    This model is a wrapper around an encoder and a decoder, and it adds a KL loss term to the reconstruction loss.

    Args:
        encoder (`nn.Module`):
            Encoder module.
        decoder (`nn.Module`):
            Decoder module.
        latent_channels (`int`, *optional*, defaults to 4):
            Number of latent channels.
    """

    def __init__(
        self,
        encoder: nn.Module,
        decoder: nn.Module,
        latent_channels: int = 4,
        dims: int = 2,
        sample_size=512,
        use_quant_conv: bool = True,
    ):
        super().__init__()

        # pass init params to Encoder
        self.encoder = encoder
        self.use_quant_conv = use_quant_conv

        # pass init params to Decoder
        quant_dims = 2 if dims == 2 else 3
        self.decoder = decoder
        if use_quant_conv:
            self.quant_conv = make_conv_nd(
                quant_dims, 2 * latent_channels, 2 * latent_channels, 1
            )
            self.post_quant_conv = make_conv_nd(
                quant_dims, latent_channels, latent_channels, 1
            )
        else:
            self.quant_conv = nn.Identity()
            self.post_quant_conv = nn.Identity()
        self.use_z_tiling = False
        self.use_hw_tiling = False
        self.dims = dims
        self.z_sample_size = 1

        self.decoder_params = inspect.signature(self.decoder.forward).parameters

        # only relevant if vae tiling is enabled
        self.set_tiling_params(sample_size=sample_size, overlap_factor=0.25)

    def set_tiling_params(self, sample_size: int = 512, overlap_factor: float = 0.25):
        self.tile_sample_min_size = sample_size
        num_blocks = len(self.encoder.down_blocks)
        self.tile_latent_min_size = int(sample_size / (2 ** (num_blocks - 1)))
        self.tile_overlap_factor = overlap_factor

    def enable_z_tiling(self, z_sample_size: int = 8):
        r"""
        Enable tiling during VAE decoding.

        When this option is enabled, the VAE will split the input tensor in tiles to compute decoding in several
        steps. This is useful to save some memory and allow larger batch sizes.
        """
        self.use_z_tiling = z_sample_size > 1
        self.z_sample_size = z_sample_size
        assert (
            z_sample_size % 8 == 0 or z_sample_size == 1
        ), f"z_sample_size must be a multiple of 8 or 1. Got {z_sample_size}."

    def disable_z_tiling(self):
        r"""
        Disable tiling during VAE decoding. If `use_tiling` was previously invoked, this method will go back to computing
        decoding in one step.
        """
        self.use_z_tiling = False

    def enable_hw_tiling(self):
        r"""
        Enable tiling during VAE decoding along the height and width dimension.
        """
        self.use_hw_tiling = True

    def disable_hw_tiling(self):
        r"""
        Disable tiling during VAE decoding along the height and width dimension.
        """
        self.use_hw_tiling = False

    def _hw_tiled_encode(self, x: torch.FloatTensor, return_dict: bool = True):
        overlap_size = int(self.tile_sample_min_size * (1 - self.tile_overlap_factor))
        blend_extent = int(self.tile_latent_min_size * self.tile_overlap_factor)
        row_limit = self.tile_latent_min_size - blend_extent

        # Split the image into 512x512 tiles and encode them separately.
        rows = []
        for i in range(0, x.shape[3], overlap_size):
            row = []
            for j in range(0, x.shape[4], overlap_size):
                tile = x[
                    :,
                    :,
                    :,
                    i : i + self.tile_sample_min_size,
                    j : j + self.tile_sample_min_size,
                ]
                tile = self.encoder(tile)
                tile = self.quant_conv(tile)
                row.append(tile)
            rows.append(row)
        result_rows = []
        for i, row in enumerate(rows):
            result_row = []
            for j, tile in enumerate(row):
                # blend the above tile and the left tile
                # to the current tile and add the current tile to the result row
                if i > 0:
                    tile = self.blend_v(rows[i - 1][j], tile, blend_extent)
                if j > 0:
                    tile = self.blend_h(row[j - 1], tile, blend_extent)
                result_row.append(tile[:, :, :, :row_limit, :row_limit])
            result_rows.append(torch.cat(result_row, dim=4))

        moments = torch.cat(result_rows, dim=3)
        return moments

    def blend_z(
        self, a: torch.Tensor, b: torch.Tensor, blend_extent: int
    ) -> torch.Tensor:
        blend_extent = min(a.shape[2], b.shape[2], blend_extent)
        for z in range(blend_extent):
            b[:, :, z, :, :] = a[:, :, -blend_extent + z, :, :] * (
                1 - z / blend_extent
            ) + b[:, :, z, :, :] * (z / blend_extent)
        return b

    def blend_v(
        self, a: torch.Tensor, b: torch.Tensor, blend_extent: int
    ) -> torch.Tensor:
        blend_extent = min(a.shape[3], b.shape[3], blend_extent)
        for y in range(blend_extent):
            b[:, :, :, y, :] = a[:, :, :, -blend_extent + y, :] * (
                1 - y / blend_extent
            ) + b[:, :, :, y, :] * (y / blend_extent)
        return b

    def blend_h(
        self, a: torch.Tensor, b: torch.Tensor, blend_extent: int
    ) -> torch.Tensor:
        blend_extent = min(a.shape[4], b.shape[4], blend_extent)
        for x in range(blend_extent):
            b[:, :, :, :, x] = a[:, :, :, :, -blend_extent + x] * (
                1 - x / blend_extent
            ) + b[:, :, :, :, x] * (x / blend_extent)
        return b

    def _hw_tiled_decode(self, z: torch.FloatTensor, target_shape):
        overlap_size = int(self.tile_latent_min_size * (1 - self.tile_overlap_factor))
        blend_extent = int(self.tile_sample_min_size * self.tile_overlap_factor)
        row_limit = self.tile_sample_min_size - blend_extent
        tile_target_shape = (
            *target_shape[:3],
            self.tile_sample_min_size,
            self.tile_sample_min_size,
        )
        # Split z into overlapping 64x64 tiles and decode them separately.
        # The tiles have an overlap to avoid seams between tiles.
        rows = []
        for i in range(0, z.shape[3], overlap_size):
            row = []
            for j in range(0, z.shape[4], overlap_size):
                tile = z[
                    :,
                    :,
                    :,
                    i : i + self.tile_latent_min_size,
                    j : j + self.tile_latent_min_size,
                ]
                tile = self.post_quant_conv(tile)
                decoded = self.decoder(tile, target_shape=tile_target_shape)
                row.append(decoded)
            rows.append(row)
        result_rows = []
        for i, row in enumerate(rows):
            result_row = []
            for j, tile in enumerate(row):
                # blend the above tile and the left tile
                # to the current tile and add the current tile to the result row
                if i > 0:
                    tile = self.blend_v(rows[i - 1][j], tile, blend_extent)
                if j > 0:
                    tile = self.blend_h(row[j - 1], tile, blend_extent)
                result_row.append(tile[:, :, :, :row_limit, :row_limit])
            result_rows.append(torch.cat(result_row, dim=4))

        dec = torch.cat(result_rows, dim=3)
        return dec

    def encode(
        self, z: torch.FloatTensor, return_dict: bool = True
    ) -> Union[DecoderOutput, torch.FloatTensor]:
        if self.use_z_tiling and z.shape[2] > self.z_sample_size > 1:
            num_splits = z.shape[2] // self.z_sample_size
            sizes = [self.z_sample_size] * num_splits
            sizes = (
                sizes + [z.shape[2] - sum(sizes)]
                if z.shape[2] - sum(sizes) > 0
                else sizes
            )
            tiles = z.split(sizes, dim=2)
            moments_tiles = [
                (
                    self._hw_tiled_encode(z_tile, return_dict)
                    if self.use_hw_tiling
                    else self._encode(z_tile)
                )
                for z_tile in tiles
            ]
            moments = torch.cat(moments_tiles, dim=2)

        else:
            moments = (
                self._hw_tiled_encode(z, return_dict)
                if self.use_hw_tiling
                else self._encode(z)
            )

        posterior = DiagonalGaussianDistribution(moments)
        if not return_dict:
            return (posterior,)

        return AutoencoderKLOutput(latent_dist=posterior)

    def _encode(self, x: torch.FloatTensor) -> AutoencoderKLOutput:
        h = self.encoder(x)
        moments = self.quant_conv(h)
        return moments

    def _decode(
        self,
        z: torch.FloatTensor,
        target_shape=None,
        timesteps: Optional[torch.Tensor] = None,
    ) -> Union[DecoderOutput, torch.FloatTensor]:
        z = self.post_quant_conv(z)
        if "timesteps" in self.decoder_params:
            dec = self.decoder(z, target_shape=target_shape, timesteps=timesteps)
        else:
            dec = self.decoder(z, target_shape=target_shape)
        return dec

    def decode(
        self,
        z: torch.FloatTensor,
        return_dict: bool = True,
        target_shape=None,
        timesteps: Optional[torch.Tensor] = None,
    ) -> Union[DecoderOutput, torch.FloatTensor]:
        assert target_shape is not None, "target_shape must be provided for decoding"
        if self.use_z_tiling and z.shape[2] > self.z_sample_size > 1:
            reduction_factor = int(
                self.encoder.patch_size_t
                * 2
                ** (
                    len(self.encoder.down_blocks)
                    - 1
                    - math.sqrt(self.encoder.patch_size)
                )
            )
            split_size = self.z_sample_size // reduction_factor
            num_splits = z.shape[2] // split_size

            # copy target shape, and divide frame dimension (=2) by the context size
            target_shape_split = list(target_shape)
            target_shape_split[2] = target_shape[2] // num_splits

            decoded_tiles = [
                (
                    self._hw_tiled_decode(z_tile, target_shape_split)
                    if self.use_hw_tiling
                    else self._decode(z_tile, target_shape=target_shape_split)
                )
                for z_tile in torch.tensor_split(z, num_splits, dim=2)
            ]
            decoded = torch.cat(decoded_tiles, dim=2)
        else:
            decoded = (
                self._hw_tiled_decode(z, target_shape)
                if self.use_hw_tiling
                else self._decode(z, target_shape=target_shape, timesteps=timesteps)
            )

        if not return_dict:
            return (decoded,)

        return DecoderOutput(sample=decoded)

    def forward(
        self,
        sample: torch.FloatTensor,
        sample_posterior: bool = False,
        return_dict: bool = True,
        generator: Optional[torch.Generator] = None,
    ) -> Union[DecoderOutput, torch.FloatTensor]:
        r"""
        Args:
            sample (`torch.FloatTensor`): Input sample.
            sample_posterior (`bool`, *optional*, defaults to `False`):
                Whether to sample from the posterior.
            return_dict (`bool`, *optional*, defaults to `True`):
                Whether to return a [`DecoderOutput`] instead of a plain tuple.
            generator (`torch.Generator`, *optional*):
                Generator used to sample from the posterior.
        """
        x = sample
        posterior = self.encode(x).latent_dist
        if sample_posterior:
            z = posterior.sample(generator=generator)
        else:
            z = posterior.mode()
        dec = self.decode(z, target_shape=sample.shape).sample

        if not return_dict:
            return (dec,)

        return DecoderOutput(sample=dec)